<a href="https://colab.research.google.com/github/illiadil/segformer_hpo/blob/main/experiment_segformer_hpo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/illiadil/segformer_hpo

Cloning into 'SegFormer-tf'...
remote: Enumerating objects: 161, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 161 (delta 14), reused 9 (delta 9), pack-reused 133
Receiving objects: 100% (161/161), 6.20 MiB | 12.41 MiB/s, done.
Resolving deltas: 100% (67/67), done.


In [ ]:
cd SegFormer-tf

/content/SegFormer-tf


In [ ]:
import numpy as np
import pandas as pd
import os
import random
import glob
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from models import SegFormer_B0

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
from sklearn.model_selection import train_test_split
from skimage.io import imread, imshow
from skimage.color import rgba2rgb

IMG_WIDTH = 640
IMG_HEIGHT = 480
IMG_CHANNELS = 3
DATASET_SIZE = 200

X = np.zeros((DATASET_SIZE, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
y = np.zeros((DATASET_SIZE, IMG_HEIGHT, IMG_WIDTH, 1), dtype=bool)

for i in range(1,DATASET_SIZE):
  image = imread("/content/drive/MyDrive/PhD/label_tool/dataset/images/"+str(i).zfill(5)+".png")[:,:,:3]
  mask_ = imread("/content/drive/MyDrive/PhD/label_tool/dataset/masks/"+str(i).zfill(5)+"_mask.png")
  X[i] = image
  mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=bool)
  mask_ = np.expand_dims(mask_, axis=-1)
  mask = np.maximum(mask, mask_[:,:,1])
  y[i] = mask

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [ ]:
checkpoint = [tf.keras.callbacks.TensorBoard(log_dir='logs')]

In [ ]:
TRAIN_STEPS = len(X_train)
VAL_STEPS = len(X_val)

optimizers = [tf.keras.optimizers.AdamW(),"Adadelta","Adagrad","Adamax","Adafactor","Nadam","Ftrl"]
for opt in optimizers:
#  model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=["accuracy"])
  model = SegFormer_B0(input_shape = (480, 640, 3), num_classes = 2)
  model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=["accuracy"])
  model.fit(X_train, y_train, validation_data=(X_val,y_val),batch_size=16, epochs=15)#, callbacks = checkpoint)
  model.save_weights("segformer_b0_"+opt+".weights.h5")
  miou = calculate_miou(model)
  print(opt+ " -> mIoU : " + miou + "%")

Epoch 1/15


KeyboardInterrupt: ignored

In [ ]:
model.save_weights("segformer_b0.weights.h5")

In [ ]:
model.load_weights("segformer_b0.weights.h5")

In [ ]:
loss = model.history.history['loss']
val_loss = model.history.history['val_loss']

plt.figure()
plt.plot( loss, 'r', label='Training loss')
plt.plot( val_loss, 'bo', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.ylim([0, 1])
plt.legend()
plt.show()

## Calculate mIoU

In [ ]:
def calculate_miou(mdl):
  threshold = 0.8
  total_iou = 0
  y_predictions = mdl.predict(X_test)
  for i in range(X_test.shape[0]):
    y_ground_truth = y_test[i]
    y_prediction = y_predictions[i][:,:,1] > threshold
    TP = (y_prediction == y_ground_truth[:,:,0])
    FN_and_FP = (y_ground_truth[:,:,0] != y_prediction)
    iou = TP.sum()/( TP.sum() + FN_and_FP.sum() )
    total_iou = total_iou + iou
  average_iou = total_iou / X_test.shape[0]*100
  return "{:.2f}".format(average_iou)
#  print("mIoU : " + "{:.2f}".format(average_iou) + "%")

In [ ]:
import random

indx = random.randint(0, X_test.shape[0])
f, axarr = plt.subplots(1,3)
axarr[0].imshow(y_predictions[indx][:,:,1] > threshold)
axarr[1].imshow(y_test[indx])
axarr[2].imshow(X_test[indx])